In [5]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.0       |           py36_0         147 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    ca-certificates-2019.8.28  |                0         132 KB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.0-py36_0     anaconda
    ca-certificates: 2019.5.15-1       --> 2019.8.28-0      anaconda
    certifi:         2019.6.16-py36_1  --> 2019.9.11-py36_0 anaconda
    openssl:         1.1.1d-h7b

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from pandas.io.html import read_html
import numpy as np

## Scrape Wikipedia Page for Table containing Postcode, Borough, Neighbourhood

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
#soup = BeautifulSoup(page, "lxml")

### Extract table from wikipedia page into dataframe

In [3]:
#print(soup.prettify())
#table_wiki = soup.find("table", class_="wikitable sortable")
df = read_html(url, attrs={"class":"wikitable sortable"})[0]
print("Shape: ", df.shape)
df.head()


Shape:  (288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Preprocessing Data

### Drop rows with borough = 'Not assigned'

In [4]:
# check for boroughs with values "Not assigned"
df_NotAssigned = df[df["Borough"]=="Not assigned"]
print("Shape: ", df_NotAssigned.shape)
df_NotAssigned.head()

Shape:  (77, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
9,M8A,Not assigned,Not assigned
13,M2B,Not assigned,Not assigned
20,M7B,Not assigned,Not assigned


In [5]:
df.drop(df[df.Borough=="Not assigned"].index, axis=0, inplace=True)
print("Shape: ", df.shape)
df.head()

Shape:  (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Replace Neighbourhoods containing  'Not assigned' with Borough name

In [6]:
# Check for neighbourhoods with 'Not assigned' values
df_NotAssigned = df[df["Neighbourhood"]=="Not assigned"]
print("Shape: ", df_NotAssigned.shape)
df_NotAssigned.head()

Shape:  (1, 3)


,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [7]:
# Replace neighbourhoods with 'Not assigned' to borough
i = (df.Neighbourhood=="Not assigned")
df.loc[i, "Neighbourhood"] = df.loc[i, "Borough"]
print("Shape: ",df.shape)
df.head(8)

Shape:  (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


### Combine neighbourhoods with the same postal codes into a comma separated list

In [29]:
df=df.groupby(["Postcode","Borough"])["Neighbourhood"].agg([("Neighbourhood", ", ".join)]).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df.shape

(103, 3)

## Obtain Geographical Coordinates for each PostCode

In [38]:
#Obtain file with geographical coordinates
!wget -O geo_data.csv https://cocl.us/Geospatial_data

--2019-09-29 11:41:31--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-09-29 11:41:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-09-29 11:41:34--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [39]:
df_geo = pd.read_csv("geo_data.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
df2 = pd.merge(left=df, right=df_geo, how="left", left_on="Postcode", right_on="Postal Code")
print("Shape: ",df2.shape)
df2.head()

Shape:  (103, 6)


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
